# Problem statement
Find the general form for constructing a DFA that can match an string representing a number in a given base that is divisible by a given divisor.

The specific case of base-3, divisible by 4 is given as a previous problem.

# Define base-3 conversion

In [1]:
def b3(n):
    if n == 0:
        return [0]
    ds = []
    while n > 0:
        ds.append(n%3)
        n = n//3
    return list(reversed(ds))

# Generate sample data

In [2]:
from collections import defaultdict, Counter
from math import log, ceil
stop = 3**4
maxlen = ceil(log(stop)/log(3))
for i in range(stop):

    args = [i,''.join(map(str,b3(i))), i%4==0, sum(b3(i)), i %12 ]
    print(*[str(x).rjust(8) for x in args ])

       0        0     True        0        0
       1        1    False        1        1
       2        2    False        2        2
       3       10    False        1        3
       4       11     True        2        4
       5       12    False        3        5
       6       20    False        2        6
       7       21    False        3        7
       8       22     True        4        8
       9      100    False        1        9
      10      101    False        2       10
      11      102    False        3       11
      12      110     True        2        0
      13      111    False        3        1
      14      112    False        4        2
      15      120    False        3        3
      16      121     True        4        4
      17      122    False        5        5
      18      200    False        2        6
      19      201    False        3        7
      20      202     True        4        8
      21      210    False        3        9
      22  

# Investigate patterns in digits

In [3]:
from pprint import pprint
def x():
    return defaultdict(lambda: 0)
# instantiate a mapping to 
results = defaultdict(dict)

for n in range(10000):
    digits = list(reversed(b3(n)))
    ds = [d for d in digits if d !=0]
    for a,b in zip(ds[:-1],ds[1:]):
        z = results[(a,b)]
        z[n%4==0] = z.get(n%4==0,0) + 1

        
# output is ordered pairs of digits and their frequency            
pprint(results)

defaultdict(<class 'dict'>,
            {(1, 1): {False: 10250, True: 3424},
             (1, 2): {False: 7488, True: 2487},
             (2, 1): {False: 8777, True: 2915},
             (2, 2): {False: 7465, True: 2500}})


I don't see much of a pattern.

# Verify answer
I have worked out my guess that there exists an extended delta function which given a current state, next character, base, and divisor can trivially return the next state of the DFA that matches strings representing numbers in the given base that are divisibly by the given divisor.

S(state, character, base, divisor) = (state\*base + character)%divisor

I need a way of implementing and simulating the DFA desribed by this function.

In [4]:
# define the aforementioned function
def state(start, character, base, divisor):
    return (int(start)*base + int(character))%divisor

In [5]:
# define a function to simulate a DFA with my state function
def traverse(s, base, divisor):
    mystate = 0
    for c in s:
        mystate = state(mystate, c, base, divisor)
    return mystate

In [6]:
# define a function for converting an magnitude to an arbitrary base
def bn(n,b):
    if n == 0:
        return [0]
    ds = []
    while n > 0:
        ds.append(n%b)
        n = n//b
    return list(reversed(ds))

In [7]:
# 11_3 % 4_10 should be 0
traverse("11",base=3,divisor=4)

0

# Test my function for a large range of numbers, bases, and divisors

In [8]:
parameters = {
    'range':(1,10000),
    'bases': (2,10),
    'divisors': (1,10),
}    

# loop over some integers
for i in range(*parameters['range']):
#     then the bases
    for base in range(*parameters['bases']):
        basen_string = ''.join(map(str,bn(i,base)))
#     then the divisors
        for divisor in range(*parameters['divisors']):
            assert traverse(basen_string,base,divisor) == i%divisor
else:
#     execute if loops exits without breaking or throwing errors
    print("Success! :)")

Success! :)


## Transition table for base-3 mod 4

In [20]:
b = 3
d = 4
l = []
for remainder in range(d):
    for digit in range(b):
        l.append([remainder,digit,state(remainder,digit,b,d)])
        print('delta({},{}) = {}'.format(*l[-1]))

delta(0,0) = 0
delta(0,1) = 1
delta(0,2) = 2
delta(1,0) = 3
delta(1,1) = 0
delta(1,2) = 1
delta(2,0) = 2
delta(2,1) = 3
delta(2,2) = 0
delta(3,0) = 1
delta(3,1) = 2
delta(3,2) = 3
